In [29]:
import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
from icecream import ic
import json
load_dotenv()


True

## load variables
- data folder
- cleaned_df
- json template
- 

In [41]:
data_folder = os.path.join(os.getcwd(),'data')
cleaned_csv = os.path.join(data_folder, 'final_range_data.csv')
cleaned_df = pd.read_csv(cleaned_csv)
halfmoon_df = pd.read_csv(os.path.join(data_folder, 'brics_csvs', 'cleaned_trees_grasses_halfmoons.csv'))

TEMPLATE_FILE = os.path.join(data_folder,"halfmoon_json_template.json")

with open(TEMPLATE_FILE, "r") as f:
    template = json.load(f)



### get names from db
the cell has two queries:
- get all somalia users
- get user details for json population

In [31]:
# create a db engine
engine = create_engine(os.getenv('DATABASE_URL1'))

#query 
query = '''select id, username, first_name, last_name, email, country from respi_regreeningusers where country in ('Somalia', 'somalia') order by first_name asc;'''
query2 = '''select id, first_name, last_name, gender, age_category, phone_number, email, country, organization, username from respi_regreeningusers where id in(3228,1831,3726,4247,4548,2495);'''
#execute the query and fetch the results
conn = engine.connect()
df = pd.read_sql_query(query, conn)
df2 = pd.read_sql_query(query2, conn)

#close the connection
conn.close()
engine.dispose()

df2

,id,first_name,last_name,gender,age_category,phone_number,email,country,organization,username
0,1831,Muktar Mohamednoor,Rage,Male,18 – 35,252616051599,mrage@so-actionagainsthunger.org,Somalia,ACF,mrage
1,2495,Hassan Omar,Khadhib,Male,,0615807566,khadhiib10@gmail.com,Somalia,Norwegian Refugee Council,Khadhib
2,3228,Jamaal,Tagal,Male,18 – 35,252616719022,suldaanyare1265@gmail.com,Somalia,ACF,JamalAli
3,3726,Abdiaziz Adan,Hassan,Male,18 – 35,0615704046,abdulazizadan02@gmail.com,Somalia,GREDO,guudow
4,4247,Abdikarim,Mohamed,Male,18 – 35,252617144970,in.qooley@gmail.com,Somalia,GREDO SOMALIA,qoley
5,4548,Abdirashid,Sheikh Mohamed,,,0615197528,abdirashid.kassim@gredosom.org,Somalia,Gredo,NRMO


## enumerator
get user details from db for json population

In [32]:
for idx, i in enumerate(df2.to_dict(orient='records'), start=1):
    i['type'] = 'ENUMERATOR'
    ic(i)

ic| i: {'age_category': '18 – 35',
        'country': 'Somalia',
        'email': 'mrage@so-actionagainsthunger.org',
        'first_name': 'Muktar Mohamednoor',
        'gender': 'Male',
        'id': 1831,
        'last_name': 'Rage',
        'organization': 'ACF',
        'phone_number': '252616051599',
        'type': 'ENUMERATOR',
        'username': 'mrage'}
ic| i: {'age_category': '',
        'country': 'Somalia',
        'email': 'khadhiib10@gmail.com',
        'first_name': 'Hassan Omar',
        'gender': 'Male',
        'id': 2495,
        'last_name': 'Khadhib',
        'organization': 'Norwegian Refugee Council',
        'phone_number': '0615807566',
        'type': 'ENUMERATOR',
        'username': 'Khadhib'}
ic| i: {'age_category': '18 – 35',
        'country': 'Somalia',
        'email': 'suldaanyare1265@gmail.com',
        'first_name': 'Jamaal',
        'gender': 'Male',
        'id': 3228,
        'last_name': 'Tagal',
        'organization': 'ACF',
        'phone_nu

## extract repeat csv records for cleaned data
1. get the 'KEY'(works like key and foreign key in the db) values from the main.csv to a list
2. For every df(repeat_csvs) get the row values where the 'PARENT_KEY' `.isin()` keys list
3. save the files

In [33]:
#step 1: get the 'KEY' values from cleaned_df to a list
keys = cleaned_df['KEY'].tolist()

#step 2.1: get the other csv filenames
dir = os.path.join(os.getcwd(),'data','brics_csvs')
main_csv = 'Rangeland_Somalia_BRCiSIII.csv'
#halfmoon_csv = 'Rangeland_Somalia_BRCiSIII-trees_grasses_halfmoons.csv'
filenames = [file for file in os.listdir(dir) if file.endswith('.csv') and file != main_csv]

#step2.2 : for every df(repeat_csvs) get the row values where the 'PARENT_KEY' `.isin()` keys list

for name in filenames:
    #read the repeat csv
    doi = pd.read_csv(os.path.join(dir,name))

    #construct its new name
    new_filename = f'cleaned_{name}'

    #gey the filtered row values for mtching keys
    filtered_doi = doi[doi['PARENT_KEY'].isin(keys)]

    #step3: save the csvs
    filtered_doi.to_csv(os.path.join(dir,'outputs',new_filename), index=False)

    print(f'💯Created: {new_filename} 🎯with {len(filtered_doi)} records')


💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_gully.csv 🎯with 0 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_contour.csv 🎯with 14 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_swales.csv 🎯with 81 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_keyline.csv 🎯with 0 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_gabions.csv 🎯with 26 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_terracing.csv 🎯with 6 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_bunds.csv 🎯with 99 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_halfmoons.csv 🎯with 212 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_basins.csv 🎯with 0 records
💯Created: cleaned_cleaned_trees_grasses_halfmoons.csv 🎯with 212 records


## farmingEntity
- read column values from cleaned df and append to json keys

In [34]:

#read and set values for farming entity
farming_entity = template[0]['farmingEntity'] 
#ic(farming_entity)

for idx, i in enumerate(cleaned_df.to_dict(orient='records'), start=1):
    #ic(i['basic_info-organisation_name'])
    farming_entity['id'] = idx
    farming_entity['first_name'] = i['basic_info-organisation_name']
    farming_entity['middle_name'] = i['basic_info-organisation_name']
    farming_entity['last_name'] = i['basic_info-organisation_name']
    farming_entity['organization'] = 'INSTITUTION'
    ic(farming_entity)

ic| farming_entity: {'first_name': 'acf',
                     'id': 1,
                     'last_name': 'acf',
                     'middle_name': 'acf',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'concern',
                     'id': 2,
                     'last_name': 'concern',
                     'middle_name': 'concern',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'concern',
                     'id': 3,
                     'last_name': 'concern',
                     'middle_name': 'concern',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'concern',
                     'id': 4,
                     'last_name': 'concern',
                     'middle_name': 'concern',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'acf',
                     'id': 5,
                     'last_name': 'acf',
                   

## plot

In [36]:
import uuid
plot_details = template[0]['plot']

plot_details_json_list = []
count = 0
#assign values from cleaned df    
for i in cleaned_df.to_dict(orient='records'):
    count += 1
    #ic(i)
    plot_details['name'] = uuid.uuid4().hex
    plot_details['calculated_size'] = i['area_ha']
    plot_details['photo_url'] = "https://radrs.icraf.org/uploaded/media/" + i['plot_details-photo']
    plot_details['farmingEntityId'] = farming_entity['id']


    ic(plot_details)
    #ic(count)
    plot_details_json_list.append(plot_details)
#ic(plot_details_json_list)



ic| plot_details: {'calculated_size': 0.469631438,
                   'conservation_area': False,
                   'crops': ['Beans', 'Sorghum', 'Orange'],
                   'farmingEntityId': 119,
                   'fence_type': [],
                   'fenced': False,
                   'has_crops': True,
                   'id': 4,
                   'livestock_allowed': False,
                   'name': '4501c6b8ecf247dfb4e0c7d5f09a1c22',
                   'other_fence_type': '',
                   'other_ownership_type': 'To be specified',
                   'photo_url': 'https://radrs.icraf.org/uploaded/media/1736568888717.jpg',
                   'plot_ownership_type': ['OTHER'],
                   'points': [{'accuracy': 2.812,
                               'altitude': 372.8,
                               'id': 10,
                               'latitude': 3.1870065,
                               'longitude': 43.4571934,
                               'plotId': 4},
    

## project

In [43]:
project = template[0]['project']
for idx, i in enumerate(cleaned_df.to_dict(orient='records'), start=1):
    project['id'] = idx
    project['project_name'] = i['basic_info-organisation_name']
    ic(project)

ic| project: {'id': 1, 'project_name': 'acf'}
ic| project: {'id': 2, 'project_name': 'concern'}
ic| project: {'id': 3, 'project_name': 'concern'}
ic| project: {'id': 4, 'project_name': 'concern'}
ic| project: {'id': 5, 'project_name': 'acf'}
ic| project: {'id': 6, 'project_name': 'concern'}
ic| project: {'id': 7, 'project_name': 'acf'}
ic| project: {'id': 8, 'project_name': 'concern'}
ic| project: {'id': 9, 'project_name': 'concern'}
ic| project: {'id': 10, 'project_name': 'concern'}
ic| project: {'id': 11, 'project_name': 'concern'}
ic| project: {'id': 12, 'project_name': 'concern'}
ic| project: {'id': 13, 'project_name': 'concern'}
ic| project: {'id': 14, 'project_name': 'concern'}
ic| project: {'id': 15, 'project_name': 'concern'}
ic| project: {'id': 16, 'project_name': 'concern'}
ic| project: {'id': 17, 'project_name': 'concern'}
ic| project: {'id': 18, 'project_name': 'concern'}
ic| project: {'id': 19, 'project_name': 'concern'}
ic| project: {'id': 20, 'project_name': 'concern'}
i

#### halfmoon csv - repeat csv
goal: get plot level information from the repeat csv
- get unique rows based on `PARENT_KEY`
- based on this, populate plot details for columns
    - `has_crops`

In [ ]:
unique_hm_df = halfmoon_df.drop_duplicates(subset=['PARENT_KEY'], keep='first')
unique_hm_df['crops_present']

0       no
1       no
2       no
7       no
10      no
11      no
12      no
13      no
14      no
17     yes
22     yes
28     yes
30     yes
38     yes
39     yes
40     yes
41     yes
42     yes
43     yes
44     yes
45     yes
46     yes
47     yes
48     yes
49     yes
50     yes
51     yes
52     yes
53     yes
54     yes
56     yes
62     yes
65     yes
68     NaN
72     NaN
75     NaN
78     NaN
118    NaN
120    NaN
124    NaN
126    NaN
128    NaN
129    NaN
134    NaN
143    NaN
163    NaN
167    NaN
170    NaN
173    NaN
176    NaN
181    NaN
189    NaN
198    NaN
206    NaN
209    NaN
Name: crops_present, dtype: object

populate the plot level info to the plot_details dictionary

- this will be based on id matching from unique(55) to main(119)

In [ ]:
count = 0
for i in unique_hm_df.to_dict(orient='records'):
    count += 1
    if i['PARENT_KEY'] in keys:
        plot_details['has_crops'] = i['crops_present']
        
        ic(plot_details)
    ic(count)

ic| plot_details: {'area': 4.908980892,
                   'calculated_size': None,
                   'conservation_area': None,
                   'crops': None,
                   'farmingEntityId': None,
                   'fence_type': None,
                   'fenced': None,
                   'has_crops': 'no',
                   'id': None,
                   'livestock_allowed': None,
                   'name': 'b886f355330e4084b1c76e6b68767a2e',
                   'other_fence_type': None,
                   'other_ownership_type': None,
                   'photo_url': 'https://radrs.icraf.org/uploaded/media/1751876154095.jpg',
                   'plot_ownership_type': None,
                   'points': None,
                   'subCounty': None,
                   'subCountyId': None}
ic| count: 1
ic| plot_details: {'area': 4.908980892,
                   'calculated_size': None,
                   'conservation_area': None,
                   'crops': None,
                